### Cele doua norme
- Norma 1: ||A||_1
- Norma infinit: ||A||_inf

In [1]:
import numpy as np

In [3]:
def norma_1(A):
    return np.max(np.sum(np.abs(A), axis=0)) # col

def norma_infinit(A):
    return np.max(np.sum(np.abs(A), axis=1)) # linie

In [4]:
A = np.array([
    [1,  2,  3],
    [-4, 5, -6]
])

print(f'norma 1: {norma_1(A)}') 
print(f'norma infinit: {norma_infinit(A)}')

norma 1: 9
norma infinit: 15


### Alegerea matricei initiale V0 
- Folosind (5) si (6)

In [5]:
def alegere_V0(A):
    norma_A_1 = norma_1(A)
    norma_A_inf = norma_infinit(A)
    AT = A.T
    V0 = AT / (norma_A_1 * norma_A_inf)
    return V0

In [6]:
alegere_V0(A)

array([[ 0.00740741, -0.02962963],
       [ 0.01481481,  0.03703704],
       [ 0.02222222, -0.04444444]])

### Metode iterative de aproximare a inversei unei matrice

1. Metoda Schultz (1)

In [7]:
def metoda_schultz(A, n, epsilon, kmax):
    V0 = alegere_V0(A)
    V1 = V0.copy()
    k = 0
    I = np.eye(n)
    
    while k < kmax:
        V0 = V1.copy()
        
        AV0 = A @ V0 # @ in loc de np.matmul()
        V1 = V0 @ (2 * I - AV0)
        
        diff = np.linalg.norm(V1 - V0, 1)
        
        if diff < epsilon:
            return V1, k, True
        
        if diff > 1e10:
            return V1, k, False
        
        k += 1
    
    return V1, k, False

In [9]:
A = np.array([
    [1, 2, 0, 0],
    [0, 1, 2, 0],
    [0, 0, 1, 2],
    [0, 0, 0, 1]
])

metoda_schultz(A, 4, 1e-10, 10000)

(array([[ 1.00000000e+00, -2.00000000e+00,  4.00000000e+00,
         -8.00000000e+00],
        [ 1.45434799e-29,  1.00000000e+00, -2.00000000e+00,
          4.00000000e+00],
        [-6.68144552e-30,  1.77156230e-29,  1.00000000e+00,
         -2.00000000e+00],
        [ 2.70202537e-30, -7.04140569e-30,  1.41875768e-29,
          1.00000000e+00]]),
 15,
 True)

2. Metoda Li (2)
- prima formula dintre cele 2 

In [11]:
def metoda_li_prima_formula(A, n, epsilon, kmax):

    V0 = alegere_V0(A)
    V1 = V0.copy()
    k = 0
    I = np.eye(n)
    
    while k < kmax:
        V0 = V1.copy()
        
        AV0 = A @ V0
        AV0_patrat = AV0 @ AV0
        V1 = V0 @ (3 * I - 3 * AV0 + AV0_patrat) # calcul putin simplificat
        
        diff = np.linalg.norm(V1 - V0, 1)
        
        if diff < epsilon:
            return V1, k, True
        
        if diff > 1e10:
            return V1, k, False
        
        k += 1
    
    return V1, k, False


In [12]:
A = np.array([
    [1, 2, 0, 0],
    [0, 1, 2, 0],
    [0, 0, 1, 2],
    [0, 0, 0, 1]
])

metoda_li_prima_formula(A, 4, 1e-10, 10000)

(array([[ 1.00000000e+00, -2.00000000e+00,  4.00000000e+00,
         -8.00000000e+00],
        [-1.83670992e-40,  1.00000000e+00, -2.00000000e+00,
          4.00000000e+00],
        [ 9.18354962e-41, -1.14794370e-41,  1.00000000e+00,
         -2.00000000e+00],
        [-4.59177481e-41,  5.73971851e-42,  2.86985925e-42,
          1.00000000e+00]]),
 10,
 True)

- a doua formula pentru Li

In [13]:
def metoda_li_a_doua_formula(A, n, epsilon, kmax):

    V0 = alegere_V0(A)
    V1 = V0.copy()
    k = 0
    I = np.eye(n)
    
    while k < kmax:
        V0 = V1.copy()
        
        V0A = V0 @ A
        temp1 = (I - V0A)
        temp2 = (3 * I - V0A)
        temp2_patrat = temp2 @ temp2

        V1 = (I + (1/4) * temp1 @ temp2_patrat) @ V0
        
        diff = np.linalg.norm(V1 - V0, 1)
        
        if diff < epsilon:
            return V1, k, True
        
        if diff > 1e10:
            return V1, k, False
        
        k += 1
    
    return V1, k, False


In [14]:
A = np.array([
    [1, 2, 0, 0],
    [0, 1, 2, 0],
    [0, 0, 1, 2],
    [0, 0, 0, 1]
])

metoda_li_a_doua_formula(A, 4, 1e-10, 10000)

(array([[ 1.00000000e+00, -2.00000000e+00,  4.00000000e+00,
         -8.00000000e+00],
        [-4.08541542e-32,  1.00000000e+00, -2.00000000e+00,
          4.00000000e+00],
        [ 1.37143848e-32, -1.08401594e-31,  1.00000000e+00,
         -2.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 9,
 True)

In [17]:
n = 4
epsilon = 1e-10
kmax = 10000

A = np.zeros((n, n))
for i in range(n):
    if i < n-1:
        A[i, i] = 1
        A[i, i+1] = 2
    else:
        A[i, i] = 1

print("Matricea A:")
print(A)          

Matricea A:
[[1. 2. 0. 0.]
 [0. 1. 2. 0.]
 [0. 0. 1. 2.]
 [0. 0. 0. 1.]]


In [18]:
print("\n1. Metoda Schultz:")

V1_schultz, iter_schultz, conv_schultz = metoda_schultz(A, n, epsilon, kmax)
if conv_schultz:
    print(f'Convergenta: {iter_schultz} iteratii')
    norma_schultz = np.linalg.norm(A @ V1_schultz - np.eye(n), 1)
    print(f'Norma: {norma_schultz}')
    print("Aproximarea inversei:")
    
    with np.printoptions(precision=6, suppress=True):
        print(V1_schultz)
else:
    print("nr maxim de iteratii atins, no good")
   


1. Metoda Schultz:
Convergenta: 15 iteratii
Norma: 1.4187576805682662e-29
Aproximarea inversei:
[[ 1. -2.  4. -8.]
 [ 0.  1. -2.  4.]
 [-0.  0.  1. -2.]
 [ 0. -0.  0.  1.]]


In [23]:
print("\n2.1 Metoda Li prima formula:")

V1_li_prima, iter_li_prima, conv_li_prima = metoda_li_prima_formula(A, n, epsilon, kmax)
if conv_li_prima:
    print(f'Convergenta: {iter_li_prima} iteratii')
    norma_li_prima = np.linalg.norm(A @ V1_li_prima - np.eye(n), 1)
    print(f'Norma: {norma_li_prima}')
    print("Aproximarea inversei:")
    
    with np.printoptions(precision=6, suppress=True):
        print(V1_li_prima)
else:
    print("nr maxim de iteratii atins, no good")
    
print("\n2.2 Metoda Li a doua formula:")

V1_li_second, iter_li_second, conv_li_second = metoda_li_a_doua_formula(A, n, epsilon, kmax)
if conv_li_second:
    print(f'Convergenta: {iter_li_second} iteratii')
    norma_li_second = np.linalg.norm(A @ V1_li_second - np.eye(n), 1)
    print(f'Norma: {norma_li_second}')
    print("Aproximarea inversei:")
    
    with np.printoptions(precision=6, suppress=True):
        print(V1_li_second)
else:
    print("nr maxim de iteratii atins, no good")
    



2.1 Metoda Li prima formula:
Convergenta: 10 iteratii
Norma: 2.220446049250313e-16
Aproximarea inversei:
[[ 1. -2.  4. -8.]
 [-0.  1. -2.  4.]
 [ 0. -0.  1. -2.]
 [-0.  0.  0.  1.]]

2.2 Metoda Li a doua formula:
Convergenta: 9 iteratii
Norma: 6.66133814775094e-16
Aproximarea inversei:
[[ 1. -2.  4. -8.]
 [-0.  1. -2.  4.]
 [ 0. -0.  1. -2.]
 [ 0.  0.  0.  1.]]


### Deducerea inductiva pt forma generala a inversei lui A

In [26]:
def forma_generala_inversa(n):
    A_inv = np.zeros((n, n))
    
    for i in range(n):
        A_inv[i, i] = 1
        for j in range(i+1, n):
            A_inv[i, j] = (-2) ** (j-i)
    
    return A_inv

print('pentru n = 4')
print(forma_generala_inversa(4))
print('pentru n = 8')
print(forma_generala_inversa(8))



pentru n = 4
[[ 1. -2.  4. -8.]
 [ 0.  1. -2.  4.]
 [ 0.  0.  1. -2.]
 [ 0.  0.  0.  1.]]
pentru n = 8
[[   1.   -2.    4.   -8.   16.  -32.   64. -128.]
 [   0.    1.   -2.    4.   -8.   16.  -32.   64.]
 [   0.    0.    1.   -2.    4.   -8.   16.  -32.]
 [   0.    0.    0.    1.   -2.    4.   -8.   16.]
 [   0.    0.    0.    0.    1.   -2.    4.   -8.]
 [   0.    0.    0.    0.    0.    1.   -2.    4.]
 [   0.    0.    0.    0.    0.    0.    1.   -2.]
 [   0.    0.    0.    0.    0.    0.    0.    1.]]


In [31]:
n = 4
print(f'folosesc n = {n}')
A_exact_inv = forma_generala_inversa(n)
with np.printoptions(precision=6, suppress=True):
    print(A_exact_inv)
    
norma_diff_schultz = np.linalg.norm(A_exact_inv - V1_schultz, 1)
norma_diff_li_prima = np.linalg.norm(A_exact_inv - V1_li_prima, 1)
norma_diff_li_second = np.linalg.norm(A_exact_inv - V1_li_second, 1)

print(f"||A_exact^(-1) - A_schultz^(-1)|| = {norma_diff_schultz}")
print(f"||A_exact^(-1) - A_li1^(-1)|| = {norma_diff_li_prima}")
print(f"||A_exact^(-1) - A_li2^(-1)|| = {norma_diff_li_second}")



folosesc n = 4
[[ 1. -2.  4. -8.]
 [ 0.  1. -2.  4.]
 [ 0.  0.  1. -2.]
 [ 0.  0.  0.  1.]]
||A_exact^(-1) - A_schultz^(-1)|| = 2.4757028733951737e-29
||A_exact^(-1) - A_li1^(-1)|| = 2.220446049250313e-16
||A_exact^(-1) - A_li2^(-1)|| = 2.220446049250314e-16


In [32]:
print("Testare:")
for test_n in [3, 4, 5, 6]:
    print(f"\nTest pentru n = {test_n}")
    
    test_A = np.zeros((test_n, test_n))
    for i in range(test_n):
        if i < test_n-1:
            test_A[i, i] = 1
            test_A[i, i+1] = 2
        else:
            test_A[i, i] = 1
    
    test_A_exact_inv = forma_generala_inversa(test_n)
    
    test_product = test_A @ test_A_exact_inv
    test_error = np.linalg.norm(test_product - np.eye(test_n), 1)
    print(f"Eroare inversa exacta: {test_error}")
    
    print("Matricea A:")
    print(test_A)
    print("Inversa exacta:")
    with np.printoptions(precision=6, suppress=True):
        print(test_A_exact_inv) 

Testare:

Test pentru n = 3
Eroarea pentru inversa exactă: 0.0
Matricea A:
[[1. 2. 0.]
 [0. 1. 2.]
 [0. 0. 1.]]
Inversa exactă:
[[ 1. -2.  4.]
 [ 0.  1. -2.]
 [ 0.  0.  1.]]

Test pentru n = 4
Eroarea pentru inversa exactă: 0.0
Matricea A:
[[1. 2. 0. 0.]
 [0. 1. 2. 0.]
 [0. 0. 1. 2.]
 [0. 0. 0. 1.]]
Inversa exactă:
[[ 1. -2.  4. -8.]
 [ 0.  1. -2.  4.]
 [ 0.  0.  1. -2.]
 [ 0.  0.  0.  1.]]

Test pentru n = 5
Eroarea pentru inversa exactă: 0.0
Matricea A:
[[1. 2. 0. 0. 0.]
 [0. 1. 2. 0. 0.]
 [0. 0. 1. 2. 0.]
 [0. 0. 0. 1. 2.]
 [0. 0. 0. 0. 1.]]
Inversa exactă:
[[ 1. -2.  4. -8. 16.]
 [ 0.  1. -2.  4. -8.]
 [ 0.  0.  1. -2.  4.]
 [ 0.  0.  0.  1. -2.]
 [ 0.  0.  0.  0.  1.]]

Test pentru n = 6
Eroarea pentru inversa exactă: 0.0
Matricea A:
[[1. 2. 0. 0. 0. 0.]
 [0. 1. 2. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0.]
 [0. 0. 0. 1. 2. 0.]
 [0. 0. 0. 0. 1. 2.]
 [0. 0. 0. 0. 0. 1.]]
Inversa exactă:
[[  1.  -2.   4.  -8.  16. -32.]
 [  0.   1.  -2.   4.  -8.  16.]
 [  0.   0.   1.  -2.   4.  -8.]
 [  0.   